In [6]:
from sklearn.model_selection import train_test_split
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, normalize, MinMaxScaler
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.decomposition import PCA
import math as m
from sklearn.naive_bayes import GaussianNB

In [2]:
def hand_norm(A):
    return m.sqrt(sum([a ** 2 for a in A]))

def hand_scalar_prod(A,B):
    return sum([a * b for a,b in zip(A,B)])

def hand_dist(A,B, metric = 'euclidean'):
    if metric == 'euclidean':
        dist = []
        for i in range(len(A)):
            aux = []
            for ii in range(len(B)):
                aux.append(m.sqrt(sum([(a - b) ** 2 for a, b in zip(A[i,:], B[ii,:])])))
            dist.append(aux)

    if metric == 'cosine':
        dist = []
        for i in range(len(A)):
            aux = []
            for ii in range(len(B)):
                aux.append(1 - (hand_scalar_prod(A[i,:],B[ii,:])/(hand_norm(A[i,:])*hand_norm(B[ii,:]))))
            dist.append(aux)
            
    if metric == 'mahalanobis':
        dist = []
        VI = np.linalg.inv(np.cov(np.vstack([A, B]).T)).T
        for i in range(len(A)):
            aux = []
            for ii in range(len(B)):
                aux.append(np.sqrt(np.dot(np.dot((A[i,:]-B[ii,:]),VI),(A[i,:]-B[ii,:]).T)))
            dist.append(aux)
            
    return np.array(dist)

In [3]:
##########################################################
# ------------------------------------------------------ #
# ----------------------- LOADING ---------------------- #
# ------------------------------------------------------ #
##########################################################
# Firstly the model loads the background and signal data, 
# then it removes the attributes first string line, which 
# are the column names, in order to avoid NaN values in 
# the array.

print('         ==== Commencing Initiation ====\n')

### Background
b_name='Input_Background_1.csv'
background = np.genfromtxt(b_name, delimiter=',')
background = background[1:,:]
Lb, W = background.shape
print("     .Background Loaded..." )
print("     .Background shape: {}".format(background.shape))

### Signal
s_name='Input_Signal_1.csv'
signal = np.genfromtxt(s_name, delimiter=',')
signal = signal[1:,:]
Ls, _ = signal.shape
print("     .Signal Loaded...")
print("     .Signal shape: {}\n".format(signal.shape))

print('\n          ==== Initiation Complete ====\n')
print('=*='*17 )
print('      ==== Commencing Data Processing ====')

         ==== Commencing Initiation ====

     .Background Loaded...
     .Background shape: (543500, 21)
     .Signal Loaded...
     .Signal shape: (522467, 21)


          ==== Initiation Complete ====

=*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*==*=
      ==== Commencing Data Processing ====


In [39]:
##########################################################
# ------------------------------------------------------ #
# --------------------- INITIATION --------------------- #
# ------------------------------------------------------ #
##########################################################
### Define User Variables ###

# List of Granularities
gra_list = [i for i in range(1,11)]

# Number of Iterations
iterations = 1

# Number of events
total = 333333

# Percentage of background samples on the testing phase
background_percent = 0.99

# Percentage of samples on the training phase
test_size = 0.3

# Percentage of background samples to divide the data-set
dat_set_percent = total/len(background)


for n_i in range(iterations):
    print('\n     => Iteration Number', (n_i+1) )

    # Divide data-set into training and testing sub-sets
    print('         .Dividing training and testing sub-sets')
    
    # Reducing background samples
    _,reduced_background = train_test_split(background, test_size=dat_set_percent)
    
    # Dividing training and test sub-sets
    background_seed, streaming_background = train_test_split(reduced_background, test_size=test_size)
    
    # Iserting the correct number of signal in training
    
    n_signal_samples = len(background_seed)*(1-background_percent)

    _,background_seed = train_test_split(background_seed, test_size=background_percent)
    
    _,training_signal = train_test_split(signal, test_size=len(background_seed)/len(signal))
    
    # Iserting the correct number of signal in streaming
    
    n_signal_samples = len(streaming_background)*(1-background_percent)

    _,streaming_background = train_test_split(streaming_background, test_size=background_percent)
    
    _,streaming_signal = train_test_split(signal, test_size=n_signal_samples/len(signal))

    # Concatenating Signal and the Background sub-sets
    
    streaming_data = np.vstack((streaming_background,streaming_signal))
    training_data = np.vstack((background_seed,training_signal))    


    print("             .Training shape: {}\n".format(training_data.shape))
    print("             .Training Background shape: {}\n".format(background_seed.shape))
    print("             .Training Signal shape: {}\n".format(training_signal.shape))
    print("             .Streaming shape: {}\n".format(streaming_data.shape))
    print("             .Streaming Background shape: {}\n".format(streaming_background.shape))
    print("             .Streaming Signal shape: {}\n".format(streaming_signal.shape))

    # Normalize Data
    print('         .Normalizing Data')
    streaming_data = normalize(streaming_data,norm='max',axis=0)
    training_data = normalize(training_data,norm='max',axis=0)

    y_training =np.ones((len(training_data)))
    y_training[:len(background_seed)] = 0
    
    y_streaming =np.ones((len(streaming_data)))
    y_streaming[:len(streaming_background)] = 0


     => Iteration Number 1
         .Dividing training and testing sub-sets
             .Training shape: (462000, 21)

             .Training Background shape: (231000, 21)

             .Training Signal shape: (231000, 21)

             .Streaming shape: (100001, 21)

             .Streaming Background shape: (99000, 21)

             .Streaming Signal shape: (1001, 21)

         .Normalizing Data


In [40]:
clf = GaussianNB(priors=[0.99,0.01])
clf.fit(training_data, y_training)

prob = clf.predict_proba(streaming_data)
print(prob.shape)

print('Background results')
print(np.min(prob[y_streaming==0,0]))
print(np.max(prob[y_streaming==0,0]))
print(np.mean(prob[y_streaming==0,0]))
print('_'*20)
print('Signal results')
print(np.min(prob[y_streaming==1,0]))
print(np.max(prob[y_streaming==1,0]))
print(np.mean(prob[y_streaming==1,0]))

(100001, 2)
Background results
2.2764804191272225e-09
1.0
0.8016296747045848
____________________
Signal results
3.099381573942637e-10
1.0
0.6990626799221598


In [42]:
def EDA_calc (data, clf, metric='euclidean', bayes=True):
    L, W = data.shape
    EDA = {}
    
    if bayes == True:
        
        prob = clf.predict_proba(data)[:,0]
        
        dist = hand_dist(data, data, metric=metric)
        
        EDA['Centrality'] = 1/np.sum(dist,axis=0)
        EDA['CumulativeProximity'] = np.sum(dist**2,axis=0)
        EDA['SquareCentrality'] = 1/EDA['CumulativeProximity']
        EDA['Eccentricity'] = 2*EDA['CumulativeProximity']/np.sum(EDA['CumulativeProximity'])
        EDA['Density'] = prob/EDA['Eccentricity']
    
    else:
        dist = hand_dist(data, data, metric=metric)

        EDA['Centrality'] = 1/np.sum(dist,axis=0)
        EDA['CumulativeProximity'] = np.sum(dist**2,axis=0)
        EDA['SquareCentrality'] = 1/EDA['CumulativeProximity']
        EDA['Eccentricity'] = 2*EDA['CumulativeProximity']/np.sum(EDA['CumulativeProximity'])
        EDA['Density'] = L/EDA['Eccentricity']
    
    return EDA

In [ ]:
streaming_eda_bayes = EDA_calc(streaming_data, clf,metric ='euclidean')
seed_eda_bayes = EDA_calc(training_data, clf,metric ='euclidean')

streaming_eda = EDA_calc(streaming_data, clf,metric ='euclidean',bayes=False)
seed_eda = EDA_calc(training_data, clf,metric ='euclidean',bayes=False)

In [ ]:
print('EDA calculated')

fig = plt.figure(figsize=[16,9*len(streaming_eda.keys())])
ax = fig.subplots(1*len(streaming_eda.keys()),1)

for i,eda in enumerate(streaming_eda):
    
    seed_var = seed_eda_bayes[eda]
    streaming_var = streaming_eda_bayes[eda]
    
    signal_streaming_var = streaming_var[y_streaming==0]
    background_streaming_var = streaming_var[y_streaming==1]

    seed_kde = stats.gaussian_kde(seed_var)
    streaming_kde = stats.gaussian_kde(streaming_var)
    signal_kde = stats.gaussian_kde(signal_streaming_var)
    background_kde = stats.gaussian_kde(background_streaming_var)

    data_eval = np.linspace(min([min(seed_var),min(streaming_var)]),
                            max([max(seed_var),max(streaming_var)]), 
                            2000)
    background_eval = np.linspace(min(background_streaming_var),
                            max(background_streaming_var), 
                            1000)
    signal_eval = np.linspace(min(signal_streaming_var),
                            max(signal_streaming_var), 
                            1000)

    # Plot the Probability Distribuction Function (PDF)
    
    ax[i].set_ylabel('Probability Density',fontsize=16)
    ax[i].set_xlabel(eda,fontsize=16)
    ax[i].set_title('Probability Distribuction Function (PDF)',fontsize=20)

    ax[i].plot(data_eval, streaming_kde(data_eval),'g', linewidth=4, label='Streaming data PDF')
    ax[i].plot(signal_eval, signal_kde(signal_eval),'r', linewidth=4, label='Streaming signal PDF')
    ax[i].plot(background_eval, background_kde(background_eval),':b', linewidth=4, 
                 label='Streaming background PDF')
    ax[i].grid()
    ax[i].legend()

plt.show()
#fig.savefig('EDA_cosine_PDF&CDF.pdf', bbox_inches='tight')

In [ ]:
streaming_eda_bayes = EDA_calc(streaming_data, clf,metric ='cosine')
seed_eda_bayes = EDA_calc(background_seed, clf,metric ='cosine')

streaming_eda = EDA_calc(streaming_data, clf,metric ='cosine',bayes=False)
seed_eda = EDA_calc(background_seed, clf,metric ='cosine',bayes=False)


print('EDA calculated')

fig = plt.figure(figsize=[16,9*len(streaming_eda.keys())])
ax = fig.subplots(1*len(streaming_eda.keys()),1)

for i,eda in enumerate(streaming_eda):
    
    seed_var = seed_eda_bayes[eda]
    streaming_var = streaming_eda_bayes[eda]
    
    signal_streaming_var = streaming_var[n_bd_str:n_bd_str+n_sl_str]
    background_streaming_var = streaming_var[:n_bd_str]

    seed_kde = stats.gaussian_kde(seed_var)
    streaming_kde = stats.gaussian_kde(streaming_var)
    signal_kde = stats.gaussian_kde(signal_streaming_var)
    background_kde = stats.gaussian_kde(background_streaming_var)

    data_eval = np.linspace(min([min(seed_var),min(streaming_var)]),
                            max([max(seed_var),max(streaming_var)]), 
                            2000)
    background_eval = np.linspace(min(background_streaming_var),
                            max(background_streaming_var), 
                            1000)
    signal_eval = np.linspace(min(signal_streaming_var),
                            max(signal_streaming_var), 
                            1000)

    # Plot the Probability Distribuction Function (PDF)
    
    ax[i].set_ylabel('Probability Density',fontsize=16)
    ax[i].set_xlabel(eda,fontsize=16)
    ax[i].set_title('Probability Distribuction Function (PDF)',fontsize=20)

    ax[i].plot(data_eval, seed_kde(data_eval), 'k', linewidth=4, label='Background seed PDF')
    ax[i].plot(data_eval, streaming_kde(data_eval),'g', linewidth=4, label='Streaming data PDF')
    ax[i].plot(signal_eval, signal_kde(signal_eval),'r', linewidth=4, label='Streaming signal PDF')
    ax[i].plot(background_eval, background_kde(background_eval),':b', linewidth=4, 
                 label='Streaming background PDF')
    ax[i].plot(data_eval, np.sqrt((seed_kde(data_eval) - streaming_kde(data_eval))**2),'-.y', 
                linewidth=4, label='Difference between\nseed and streaming PDF')
    ax[i].grid()
    ax[i].legend()

plt.show()
#fig.savefig('EDA_cosine_PDF&CDF.pdf', bbox_inches='tight')

In [ ]:
streaming_eda_bayes = EDA_calc(streaming_data, clf,metric ='mahalanobis')
seed_eda_bayes = EDA_calc(background_seed, clf,metric ='mahalanobis')

streaming_eda = EDA_calc(streaming_data, clf,metric ='mahalanobis',bayes=False)
seed_eda = EDA_calc(background_seed, clf,metric ='mahalanobis',bayes=False)


print('EDA calculated')

fig = plt.figure(figsize=[16,9*len(streaming_eda.keys())])
ax = fig.subplots(1*len(streaming_eda.keys()),1)

for i,eda in enumerate(streaming_eda):
    
    seed_var = seed_eda_bayes[eda]
    streaming_var = streaming_eda_bayes[eda]
    
    signal_streaming_var = streaming_var[n_bd_str:n_bd_str+n_sl_str]
    background_streaming_var = streaming_var[:n_bd_str]

    seed_kde = stats.gaussian_kde(seed_var)
    streaming_kde = stats.gaussian_kde(streaming_var)
    signal_kde = stats.gaussian_kde(signal_streaming_var)
    background_kde = stats.gaussian_kde(background_streaming_var)

    data_eval = np.linspace(min([min(seed_var),min(streaming_var)]),
                            max([max(seed_var),max(streaming_var)]), 
                            2000)
    background_eval = np.linspace(min(background_streaming_var),
                            max(background_streaming_var), 
                            1000)
    signal_eval = np.linspace(min(signal_streaming_var),
                            max(signal_streaming_var), 
                            1000)

    # Plot the Probability Distribuction Function (PDF)
    
    ax[i].set_ylabel('Probability Density',fontsize=16)
    ax[i].set_xlabel(eda,fontsize=16)
    ax[i].set_title('Probability Distribuction Function (PDF)',fontsize=20)

    ax[i].plot(data_eval, seed_kde(data_eval), 'k', linewidth=4, label='Background seed PDF')
    ax[i].plot(data_eval, streaming_kde(data_eval),'g', linewidth=4, label='Streaming data PDF')
    ax[i].plot(signal_eval, signal_kde(signal_eval),'r', linewidth=4, label='Streaming signal PDF')
    ax[i].plot(background_eval, background_kde(background_eval),':b', linewidth=4, 
                 label='Streaming background PDF')
    ax[i].plot(data_eval, np.sqrt((seed_kde(data_eval) - streaming_kde(data_eval))**2),'-.y', 
                linewidth=4, label='Difference between\nseed and streaming PDF')
    ax[i].grid()
    ax[i].legend()

plt.show()
#fig.savefig('EDA_cosine_PDF&CDF.pdf', bbox_inches='tight')

In [77]:
prob = np.ones((len(streaming_data)))
for i in range(len(streaming_data[0,:])):
    kde = stats.gaussian_kde(background_seed[:,i])
    prob *= kde(streaming_data[:,i])
prob *= 0.99



print('Background results')
print(np.min(prob[y==0]))
print(np.max(prob[y==0]))
print(np.mean(prob[y==0]))
print('_'*20)
print('Signal results')
print(np.min(prob[y==1]))
print(np.max(prob[y==1]))
print(np.mean(prob[y==1]))

Background results
1.2468138366805565e-14
6783962.284219568
157193.3387023344
____________________
Signal results
4.99595314771325
51919.312730448575
13011.263171681605
